Setup

In [1]:
import itertools
import numpy as np
from numpy.random import *
import graph_tool.all as gt

import time

from multiprocessing import Pool

import pandas as pd

import os

import pickle

n = 100000

p = 512 / 99999
#d = 512
d = (int) (p * (n-1))
t = 1
k = (int) (d * 32/512)
k_adj = 2
k_m = k * k_adj

#r_approx = 5420
r_approx = (int) ((n / 100000) * (pow(2, 16) * k / d))

#if n >= 10^5:
#    p = d / n
#else:
#    p = d / (n - 1)

print("N : {}".format(n))
print("D : {}".format(d))
print("K : {}".format(k_m))
print("R : {}".format(r_approx))



rng = np.random.default_rng(seed=42)


def add_fast_gnp_edges():
    num_edges = rng.binomial(n*(n-1)/2, p)
    sources = rng.integers(0, n, num_edges*2)
    targets = rng.integers(0, n, num_edges*2)
    mask = sources != targets # removes self-loops
    g.add_edge_list(np.column_stack((sources[mask], targets[mask])))

g = gt.Graph(directed=True)
g.add_vertex(n)
add_fast_gnp_edges()

g.vp["mode_T"] = g.new_vp("int")
g.vp["mode_q"] = g.new_vp("int")
g.vp["mode_f"] = g.new_vp("int")

g.vp["mode_T"].a = t
g.vp["mode_q"].a = 1
g.vp["mode_f"].a = 0

g.ep["mode_w"] = g.new_ep("double")
g.ep["mode_qq"] = g.new_ep("int")

g.ep["mode_w"].a = 0
g.ep["mode_qq"].a = 1



global trackMinK
global trackMaxK
trackMinK = 100
trackMaxK = 0

global maxInterference
maxInterference = 0.0





def fast_sum_weights(s_i):
    W = g.get_in_edges(s_i, [g.ep["mode_w"]])[:,2]
    F = np.array(g.get_in_neighbors(s_i, [g.vp["mode_f"]])[:,1], dtype=bool)
    return W[F].sum()

def update_graph_SJOIN():
    B = []
    for s_i in g.iter_vertices():
        w_i = fast_sum_weights(s_i)
        if w_i > g.vp["mode_T"][s_i]:
            g.vp["mode_q"][s_i] = 2
            B.append(s_i)
    return B


def fast_sum_weights_multi(s_i):
    W = g.get_in_edges(s_i, [g.ep["mode_w"]])[:,2]
    F = np.array(g.get_in_neighbors(s_i, [g.vp["mode_f"]])[:,1], dtype=bool)
    return W[F].sum()

def update_graph_SJOIN_multi():
    B = []
    V = g.get_vertices()

    with Pool() as pool:
        Ws = pool.map(fast_sum_weights_multi, V)

    for i in range(len(V)):

        w_i = Ws[i]

        if w_i > g.vp["mode_T"][V[i]]:
            g.vp["mode_q"][V[i]] = 2
            B.append(V[i])
    return B


def SJOIN_one_step(A):
    global trackMinK
    global trackMaxK

    #n = 100000
    min_k = 15
    max_k = 45

    #n = 50000
    #min_k = 10
    #max_k = 24

    #n = 20000
    #min_k = 1
    #max_k = 15

    min_memory = None
    max_memory = None
    while True:
        mid_k = round((min_k+max_k)/2)
        for i in A:
            g.vp["mode_f"][i] = 1
            g.ep["mode_w"].a[g.get_out_edges(i, [g.edge_index])[:,2]] = (t / mid_k) + 1e-10
        
        
        
        #B = update_graph_SJOIN()
        B = update_graph_SJOIN_multi()

        #print("({} - {}) {} : {}".format(min_k, max_k, mid_k, len(B)))


        g.vp["mode_f"].a = 0
        g.vp["mode_q"].a = 1

        if len(B) > r_approx:
            min_k = mid_k+1
            max_memory = B
        else:
            max_k = mid_k-1
            min_memory = B

        if max_k<min_k:

            #print("{}, {}".format(len(min_memory), len(max_memory)))
            if len(max_memory)-r_approx > r_approx-len(min_memory):
                B = min_memory
                mid_k = max_k
            else:
                B = max_memory
                mid_k = min_k
            #if track_weights:
            #    for i in A:
            #        mode_w.a[g.get_out_edges(i, [g.edge_index])[:,2]] = (t / k_m) * scale
            #        out_edges_i = g.get_out_edges(i, [g.edge_index])
            #        neighbors_not_B = np.setdiff1d(out_edges_i[:,1], B)
            #        edges_to_deactivate = out_edges_i[np.isin(out_edges_i[:,1],neighbors_not_B)]
            #        mode_w.a[edges_to_deactivate[:,2]] = 0
            print("{} : {}".format(mid_k, len(B)))
            if mid_k > trackMaxK:
                trackMaxK = mid_k
            if mid_k < trackMinK:
                trackMinK = mid_k
            return B


def SJOIN_sequence(A, L):
    S = [A]
    for i in range(L-1):
        B = SJOIN_one_step(S[-1])
        S.append(B)
    return S

def SJOIN_interference_check(M, A_i, i):
    global maxInterference
    for B_i in range(i):
        A = M[A_i]
        B = M[B_i]
        intersect = len(set(A) & set(B))
        interference = intersect / len(A)
        if interference > maxInterference:
            maxInterference = interference
        if interference >= 1/2:
            return True
    return False

def SJOIN_simulation(L, num):
    global maxInterference
    
    g.load("brain.graphml")

    state = np.load("rng.npy" ,allow_pickle='TRUE').item()
    rng.bit_generator.__setstate__(state)

    with open('M.pkl', 'rb') as f:
        M = pickle.load(f)

    df = None

    if os.path.isfile('output.xlsx'):
        print("File found")
        df = pd.read_excel('output.xlsx')
        maxInterference = df.loc[len(df.index)-1][0]
    else:
        print("Create new file")
        columns = ["Interference"]
        for i in range(L):
            columns.append("Memory {}".format(i+1))
        df = pd.DataFrame(columns=columns)
        maxInterference = 0

    sequence_count = 1
    while True:
        print("Sequence {}".format(sequence_count))
        
        A = rng.choice(np.arange(0,n-1), size=r_approx)
        S = SJOIN_sequence(A, L)
        M.extend(S)
        
        if SJOIN_interference_check(M, len(M)-len(S), len(M)-len(S)):
            df.to_excel('output.xlsx', index=False)
            return M
        for B_i in range(len(M)-len(S)+1, len(M)):
            if SJOIN_interference_check(M, B_i, B_i-1):
                df.to_excel('output.xlsx', index=False)
                return M
        
        new_row = [maxInterference]
        for s in S:
            new_row.append(len(s))

        df.loc[len(df.index)] = new_row

        if sequence_count >= num:
            df.to_excel('output.xlsx', index=False)
            g.save("brain.graphml")
            np.save("rng.npy", rng.bit_generator.state)
            with open('M.pkl', 'wb') as f:
                pickle.dump(M, f)
            return M
        
        sequence_count = sequence_count+1

def SJOIN_long_sequence(num):
    global maxInterference

    g.load("brain.graphml")

    state = np.load("rng.npy" ,allow_pickle='TRUE').item()
    rng.bit_generator.__setstate__(state)

    with open('M.pkl', 'rb') as f:
        M = pickle.load(f)

    df = None

    if os.path.isfile('output.xlsx'):
        print("File found")
        df = pd.read_excel('output.xlsx')
        maxInterference = df.loc[len(df.index)-1][0]
    else:
        print("Create new file")
        columns = ["Interference", "Memory"]
        df = pd.DataFrame(columns=columns)
        maxInterference = 0

    if len(M) == 0:
        M.append(rng.choice(np.arange(0,n-1), size=r_approx))
        new_row = [0, r_approx]
        df.loc[len(df.index)] = new_row

    memory_count = 1
    while True:
        print("Memory {}".format(memory_count))

        A = M[-1]
        B = SJOIN_one_step(A)
        M.append(B)

        if SJOIN_interference_check(M, len(M)-1, len(M)-2):
            df.to_excel('output.xlsx', index=False)
            return M
        
        new_row = [maxInterference, len(B)]
        df.loc[len(df.index)] = new_row

        if memory_count >= num:
            df.to_excel('output.xlsx', index=False)
            g.save("brain.graphml")
            np.save("rng.npy", rng.bit_generator.state)
            with open('M.pkl', 'wb') as f:
                pickle.dump(M, f)
            return M
        
        memory_count = memory_count + 1
        


    


N : 100000
D : 512
K : 64
R : 4096


Create New Graph

In [3]:
rng = np.random.default_rng(seed=42)

g = gt.Graph(directed=True)
g.add_vertex(n)
add_fast_gnp_edges()

g.vp["mode_T"] = g.new_vp("int")
g.vp["mode_f"] = g.new_vp("int")

g.vp["mode_T"].a = t
g.vp["mode_f"].a = 0

g.ep["mode_w"] = g.new_ep("double")
g.ep["mode_qq"] = g.new_ep("int")

g.ep["mode_w"].a = 0
g.ep["mode_qq"].a = 1

M = []


g.save("brain.graphml")

np.save("rng.npy", rng.bit_generator.state)

with open('M.pkl', 'wb') as f:
    pickle.dump(M, f)

Execution

In [2]:
global trackMinK
global trackMaxK


#M = SJOIN_simulation(5, 50)
M = SJOIN_long_sequence(1000)
print("{} : {}".format(trackMinK, trackMaxK))

File found
Memory 1


/tmp/ipykernel_620/1489839546.py:279: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  maxInterference = df.loc[len(df.index)-1][0]


30 : 4597
Memory 2
32 : 3779
Memory 3
27 : 3784
Memory 4
27 : 3667
Memory 5
28 : 4291
Memory 6
30 : 4032
Memory 7
30 : 4675
Memory 8
34 : 4483
Memory 9
33 : 4202
Memory 10
31 : 4690
Memory 11
34 : 4639
Memory 12
34 : 4161
Memory 13
31 : 4372
Memory 14
32 : 4858
Memory 15
35 : 4786
Memory 16
33 : 3967
Memory 17
28 : 3943
Memory 18
28 : 3799
Memory 19
27 : 3929
Memory 20
28 : 3700
Memory 21
28 : 4734
Memory 22
33 : 3564
Memory 23
26 : 3235
Memory 24
24 : 3192
Memory 25
25 : 4464
Memory 26
31 : 4007
Memory 27
30 : 4555
Memory 28
32 : 3359
Memory 29
26 : 4470
Memory 30
33 : 4223
Memory 31
30 : 3353
Memory 32
26 : 4335
Memory 33
32 : 4437
Memory 34
31 : 3865
Memory 35
29 : 4785
Memory 36
33 : 3959
Memory 37
28 : 3987
Memory 38
30 : 4179
Memory 39
31 : 4444
Memory 40
31 : 3780
Memory 41
27 : 3824
Memory 42
29 : 4246
Memory 43
30 : 3589
Memory 44
26 : 3509
Memory 45
27 : 4405
Memory 46
31 : 3531
Memory 47
27 : 4621
Memory 48
32 : 3898
Memory 49
28 : 3380
Memory 50
26 : 4759
Memory 51
33 : 370

In [7]:
global maxInterference

M = []

while True:

    A = rng.choice(np.arange(0,n-1), size=r_approx)
    M.append(A)

    if len(M) > 1 and SJOIN_interference_check(M, len(M)-1, len(M)-1):
        break

    print("{} : {:.5f}".format(len(M), maxInterference))

1 : 0.00000
2 : 0.03243
3 : 0.03632
4 : 0.03891
5 : 0.04150
6 : 0.04280
7 : 0.04280
8 : 0.04799
9 : 0.05058
10 : 0.05058
11 : 0.05188
12 : 0.05188
13 : 0.05188
14 : 0.05837
15 : 0.05837
16 : 0.05837
17 : 0.05837
18 : 0.05837
19 : 0.05837
20 : 0.05837
21 : 0.05837
22 : 0.05837
23 : 0.05837
24 : 0.05837
25 : 0.05837
26 : 0.05837
27 : 0.05837
28 : 0.05837
29 : 0.05837
30 : 0.05837
31 : 0.05837
32 : 0.05837
33 : 0.05837
34 : 0.05837
35 : 0.05837
36 : 0.05837
37 : 0.05837
38 : 0.05837
39 : 0.05837
40 : 0.05837
41 : 0.05837
42 : 0.05837
43 : 0.05837
44 : 0.05837
45 : 0.05837
46 : 0.05837
47 : 0.05837
48 : 0.05837
49 : 0.05837
50 : 0.05837
51 : 0.06874
52 : 0.06874
53 : 0.06874
54 : 0.06874
55 : 0.06874
56 : 0.06874
57 : 0.06874
58 : 0.06874
59 : 0.06874
60 : 0.06874
61 : 0.06874
62 : 0.06874
63 : 0.06874
64 : 0.06874
65 : 0.06874
66 : 0.06874
67 : 0.06874
68 : 0.06874
69 : 0.06874
70 : 0.06874
71 : 0.06874
72 : 0.06874
73 : 0.06874
74 : 0.06874
75 : 0.06874
76 : 0.06874
77 : 0.06874
78 : 0.0

KeyboardInterrupt: 

In [19]:
import math


n = 20000
m1 = 1183
m2 = 352

p = m1/n
s = m2 - (int) (m2/2)


est = math.comb(m2, s) * math.pow((1-p), (m2-s))

for i in range(s):
    est = est * p

print(est)

6.230789522974971e-117
